In [35]:
import numpy as np
X = {}
y = {}
X["train"] = np.load("./train_feature_save.npy",allow_pickle=True )[:9025]
y["train"] = np.load("./train_label_save.npy")[:9025]
X["dev"] = np.load("./test_feature_save.npy")[:1003]
y["dev"] = np.load("./test_label_save.npy")[:1003]
X["train"].shape, X["dev"].shape

((9025, 768), (1003, 768))

In [84]:
np.sum(y["dev"])/y["dev"].shape[0], np.sum(y["train"])/y["train"].shape[0]

(0.5264207377866401, 0.535623268698061)

In [38]:
y["train"]=y["train"].squeeze()
y["dev"]=y["dev"].squeeze()
y["train"].shape, y["dev"].shape

((9025,), (1003,))

In [64]:
from sklearn.linear_model import LogisticRegression
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2)

model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])

0.8005982053838484

In [65]:
import sklearn.metrics as mc
predl = model.predict(X["dev"])
mc.f1_score(y["dev"], predl)

0.8154981549815499

In [66]:
fpr, tpr, thresholds = mc.roc_curve(y["dev"], pred)
mc.auc(fpr, tpr)

0.878975279106858

In [67]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((9025, 769), (1003, 769))

In [68]:
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0]
np.all(np.linalg.eigvals(hessian) > 0)

True

In [69]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [70]:
np.all(np.linalg.eigvals(hessian) > 0)

True

In [71]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1003, 9025)

In [72]:
np.save("old_predictions.npy", pred)

In [73]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    
    #print("prediction", prediction)
    

    return X_k, y_k, prediction

In [74]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=0.1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    change = -(model.predict_proba(test_point)[0][1] - new)
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = (model.predict(test_point) == model_k.predict(test_point))
    
    """
    print("change    ", change)
    print("old       ", model.predict_proba(test_point)[0][1])
    print()
    """
    
  
    return change, flip, prediction,new

# exact_k

In [75]:
def exact_k(test_idx):
    for k in range(1, np.sum(y["train"])):
        _, flip,_ = new_train(k, test_idx, delta_pred)
        if flip == False:
            return k
    return None

In [76]:
"""
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

exact_ks = []
for test_idx in range(X["dev"].shape[0]):
    k =exact_k(test_idx)
    print(test_idx, k)
    exact_ks.append(k)

exact_ks=np.array(exact_ks)
np.save("exact_ks_IP.npy", exact_ks)
"""

'\n# Loop over all dev points:\nimport warnings\nwarnings.filterwarnings("ignore")\n\nexact_ks = []\nfor test_idx in range(X["dev"].shape[0]):\n    k =exact_k(test_idx)\n    print(test_idx, k)\n    exact_ks.append(k)\n\nexact_ks=np.array(exact_ks)\nnp.save("exact_ks_IP.npy", exact_ks)\n'

# Find approximate k by IF

In [77]:
def pred_change(k, scores, test_idx):
    X_k, y_k, prediction = Remove(k, delta_pred, test_idx)
    

In [78]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    
    if pred[test_idx] > 0.5:
        top_k_index = np.flip(delta_pred[test_idx].argsort())
    else:
        top_k_index = delta_pred[test_idx].argsort()
    
    for k in range(1, y["train"].shape[0]):
        change = -np.sum(delta_pred[test_idx][top_k_index[:k]])
        #print("prediction", change)
        
        if old > 0.5 and old + change < 0.5:
            return k
        elif old < 0.5 and old + change > 0.5:
            return k
        
    return None

In [79]:
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

appro_ks = []
new_predictions = []
for test_idx in range(X["dev"].shape[0]):
    appro_k = approximate_k(test_idx)
    if appro_k != None:
        _, _, _, new_prediction = new_train(appro_k, test_idx, delta_pred)
        print(test_idx, appro_k)
        print("old", pred[test_idx], "new", new_prediction)
        print()
        appro_ks.append(appro_k)
        new_predictions.append(new_prediction)
    else:
        appro_ks.append(None)
        new_predictions.append(None)



0 19
old [0.56084799] new 0.3851226795151839

1 2708
old [0.89775399] new 1.5395254489435875e-06

2 458
old [0.20004584] new 0.9360332010195475

4 897
old [0.18471193] new 0.9655392123840821

5 32
old [0.57088033] new 0.3749256903854698

7 24
old [0.42589126] new 0.6026129307338145

8 786
old [0.14115718] new 0.9990018331181981

9 101
old [0.69467775] new 0.4194523884425906

10 27
old [0.43148249] new 0.6519855133038086

11 486
old [0.82124758] new 0.014395069578203018

13 236
old [0.21992586] new 0.7919872492863356

14 312
old [0.22139425] new 0.8743926435573388

15 290
old [0.73870708] new 0.18500741246783345

16 1099
old [0.86503494] new 0.0007846574508554796

17 653
old [0.85936252] new 0.005115634002873737

18 390
old [0.77262801] new 0.06969502017012459

19 41
old [0.42564786] new 0.45582935279810205

20 644
old [0.1643815] new 0.9959233048547119

21 414
old [0.11672637] new 0.9998104614464621

22 779
old [0.84222642] new 0.008204090228318325

23 118
old [0.70752099] new 0.323540

200 65
old [0.37065948] new 0.6573545839161664

201 222
old [0.2705415] new 0.8149099312492755

202 620
old [0.83868844] new 0.013208941364379724

203 71
old [0.38464947] new 0.5264608141074812

204 46
old [0.58588559] new 0.4794748833053938

205 73
old [0.63197113] new 0.17317989971824865

206 2
old [0.49274172] new 0.47749362154364383

208 1887
old [0.13051988] new 0.9999646946566411

210 8
old [0.47945106] new 0.5925948332506841

211 25
old [0.43611221] new 0.48142342511134256

213 637
old [0.84422003] new 0.01810459492707479

214 105
old [0.66645543] new 0.32526606419605486

216 183
old [0.71216747] new 0.28801420110265696

217 116
old [0.27562821] new 0.6786238769192301

218 335
old [0.23568461] new 0.8252459960589643

219 180
old [0.31937747] new 0.6950692448889245

221 16
old [0.54114588] new 0.6176597725777364

223 98
old [0.3628524] new 0.5087734171987542

224 101
old [0.69193872] new 0.23114508132442046

225 407
old [0.79965787] new 0.05541171999734026

227 764
old [0.1641690

402 3387
old [0.90132448] new 5.956955661827794e-08

403 466
old [0.83622518] new 0.0165657857101672

404 15
old [0.54416862] new 0.32288566743199204

405 11
old [0.53023864] new 0.36767730531104253

406 1903
old [0.90024537] new 1.3408900692586612e-06

408 353
old [0.81469592] new 0.014731975967561773

409 622
old [0.18355792] new 0.9335266579308849

410 3164
old [0.89775894] new 6.871737875462241e-08

411 28
old [0.42316055] new 0.6740773612805742

412 416
old [0.79391241] new 0.09225460466512321

413 618
old [0.24324634] new 0.904230028284105

414 16
old [0.56443662] new 0.6438579785472645

415 2858
old [0.11409917] new 0.9999994268677307

416 2
old [0.48887561] new 0.6460611768913467

417 74
old [0.36742046] new 0.6208560776658607

418 107
old [0.62432492] new 0.34882120428756974

419 216
old [0.7479761] new 0.19010313715354643

420 1266
old [0.1117905] new 0.9999558212948847

421 193
old [0.25128591] new 0.5047075654269768

422 89
old [0.66256596] new 0.47440786048792694

423 1
ol

597 955
old [0.17347678] new 0.977180620677722

598 358
old [0.26384026] new 0.8042744510118163

599 1055
old [0.86667105] new 0.0006494692012154415

601 107
old [0.65737732] new 0.44943812800479493

603 133
old [0.67930431] new 0.2297048878444815

604 3
old [0.51148187] new 0.4245041476958816

605 9
old [0.46647574] new 0.4897285809673469

607 1138
old [0.86139482] new 0.0006657898746393789

608 403
old [0.75540889] new 0.1519122933530173

609 22
old [0.44914209] new 0.5480902253825771

610 313
old [0.74568429] new 0.1610694602324647

611 82
old [0.6493779] new 0.4732608049188583

612 372
old [0.19858536] new 0.9541399671865557

613 127
old [0.75862749] new 0.4073208192027568

616 39
old [0.56533253] new 0.48452953054516174

617 328
old [0.78328327] new 0.12661881209954845

618 160
old [0.68174864] new 0.32816969087236486

619 356
old [0.25048213] new 0.8529465966997174

620 100
old [0.35334634] new 0.6047800216986059

621 245
old [0.21191296] new 0.868228327148925

622 302
old [0.794

803 2269
old [0.89448635] new 1.9310231928818486e-06

804 273
old [0.75425897] new 0.23108296614001936

805 4218
old [0.07582026] new 0.9999999999979947

806 109
old [0.31030237] new 0.4192649029636337

807 589
old [0.839723] new 0.02003298058923874

808 522
old [0.2090067] new 0.9727507442322766

809 18
old [0.46021016] new 0.5976980566740784

810 1001
old [0.83510531] new 0.007109015689186866

812 342
old [0.75502587] new 0.06434778157103606

813 744
old [0.18293065] new 0.9867824805595914

814 2
old [0.50594233] new 0.5489641564825197

815 53
old [0.60571959] new 0.44402050795719866

816 32
old [0.56531567] new 0.4152733546728115

817 675
old [0.18065512] new 0.9849535503704373

818 3785
old [0.1182738] new 0.9999996651567673

819 3
old [0.48515681] new 0.3631181860892114

820 47
old [0.409116] new 0.4852447113255701

821 26
old [0.58543158] new 0.38540571658136347

822 525
old [0.22642819] new 0.9118948694964243

824 56
old [0.61545214] new 0.47412279337247454

825 566
old [0.19915

In [81]:
appro_ks= np.array(appro_ks)
new_predictions=np.array(new_predictions)
np.save("appro_ks_IP.npy", appro_ks)
np.save("new_predictions.npy", new_predictions)
np.save("old_predictions.npy", pred)

# Difference of change and prediction when k increasing

In [ ]:
"""
row = len(list(range(0, X["dev"].shape[0])))
col = len(list(range(1, np.sum(y["train"]))))
changes = np.zeros((row, col))
prediction = np.zeros((row, col))

import warnings
warnings.filterwarnings("ignore")
for i in range(0, X["dev"].shape[0]):
    print(i)
    for j in range(1, np.sum(y["train"]):
        changes[i, j-1], _, prediction[i, j-1] = new_train(j, i, delta_pred)
        
   
np.save("pred_change_app.npy", prediction)
np.save("changes.npy", changes)

difference = prediction - changes
np.save("difference_k_appk.npy", difference)
"""

In [82]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_emo'